# Preliminary Racquet Features EDA
In this notebook, I do a preliminary overview of the racquet data I scraped from TW's website. In particular I take note of any outlier columns and potential data type issues.

## Table of Contents
1. [Load data and basic snapshot](#load-data-and-basic-snapshot)

2. [Examining object columns that need to be converted to float or int](#examining-object-columns-that-need-to-be-converted-to-float-or-int)

3. [Looking at how junior racquets affect the data](#looking-at-how-junior-racquets-affect-the-data)

4. [Takeaways and next steps](#takeaways-and-next-steps)

In [41]:
# Imports
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns

## Load data and basic snapshot
In this section, I run the basic EDA functions on the data to get an idea of what it actually looks like. I take note of any oddities or potential issues as I look at each output.

In [42]:
# Load in raw data from csv file
racquets = pd.read_csv("../data/raw/racquet_features_raw.csv", index_col = 0)

It might be good to extract the racquet brand name for meta data when I go to create embeddings for each racquet.

In [43]:
# Get a look at what the rows of the data actually looks like
racquets.head()

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.8,289.0,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.0,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.0,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.0,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.0,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I look at the shape of the df below. It has 397 rows and **37** columns. This is odd because I did not have 37 columns in the dictionary I built when writing the scraper. My initial guess is that something unexpected happened with the spec tables because that was the section that was causing me the most trouble when scraping.

In [44]:
# Look at the shape of the data
racquets.shape

(397, 37)

Below is a list of each column, its non-null count, and its data type. Interestingly there is a group of columns (20 to 31) that are clearly duplicated and have all null values.

In [45]:
# List data types of each column and look at NA values
racquets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397 entries, 0 to 0
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   racquet_img      397 non-null    object 
 1   racquet_name     397 non-null    object 
 2   racquet_rating   343 non-null    float64
 3   racquet_price    397 non-null    float64
 4   racquet_desc     397 non-null    object 
 5   Head Size        365 non-null    object 
 6   Length           365 non-null    object 
 7   Strung Weight    307 non-null    object 
 8   Balance          309 non-null    object 
 9   Swingweight      309 non-null    float64
 10  Stiffness        308 non-null    object 
 11  Beam Width       309 non-null    object 
 12  Composition      364 non-null    object 
 13  Power Level      309 non-null    object 
 14  Stroke Style     309 non-null    object 
 15  Swing Speed      309 non-null    object 
 16  Racquet Colors   353 non-null    object 
 17  Grip Type        309 no

Looking at the list of column names further confirms the reality that the spec columns were duplicated for some reason. However, since the rest of the data seems to be in good condition, I think it would just be easier to remove the duplicated columns rather than trying to edit the scraper.

In [46]:
# Look at list of column names - specs seemed to be duplicated?
racquets.columns

Index(['racquet_img', 'racquet_name', 'racquet_rating', 'racquet_price',
       'racquet_desc', 'Head Size', 'Length', 'Strung Weight', 'Balance',
       'Swingweight', 'Stiffness', 'Beam Width', 'Composition', 'Power Level',
       'Stroke Style', 'Swing Speed', 'Racquet Colors', 'Grip Type',
       'String Pattern', 'String Tension', 'Balance:', 'Swingweight:',
       'Stiffness:', 'Beam Width:', 'Composition:', 'Power Level:',
       'Stroke Style:', 'Swing Speed:', 'Racquet Colors:', 'Grip Type:',
       'String Pattern:', 'String Tension:', 'Age', 'Weight', 'Height',
       'Other', 'Strung  Weight'],
      dtype='object')

## Examining object columns that need to be converted to float or int

Below is the descriptive statistics for the quantiative columns. Notice that some columns that **should** be quantitative, like "Head Size" are not shown. This is because I recorded the spec values as strings (with their units) when scraping TW. I will need to process those columns to convert them into float data types. This is confirmed by the .describe() function's output for the object columns, which include "Head Size", "Length", "Balance", etc.

In [47]:
# Basic descriptive stats
racquets.describe()

,racquet_rating,racquet_price,Swingweight,Balance:,Swingweight:,Stiffness:,Beam Width:,Composition:,Power Level:,Stroke Style:,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:
count,343.000000,397.000000,309.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,4.730612,198.241033,317.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.471989,87.496613,11.11364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,13.950000,270.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,4.600000,134.000000,310.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4.900000,199.000000,318.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.000000,269.000000,325.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,5.000000,579.000000,345.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
racquets.describe(include = ["object"])

,racquet_img,racquet_name,racquet_desc,Head Size,Length,Strung Weight,Balance,Stiffness,Beam Width,Composition,...,Swing Speed,Racquet Colors,Grip Type,String Pattern,String Tension,Age,Weight,Height,Other,Strung Weight
count,397,397,397,365,365,307,309,308,309,364,...,309,353,309,312,306,56,3,3,5,2
unique,394,389,391,37,25,34,78,22,91,55,...,6,83,45,15,23,9,3,3,4,1
top,https://img.tennis-warehouse.com/watermark/rs....,Wilson Pro Staff Six.One 95 v14,Introducing the Wilson Ultra Pro 18x20 v4! O...,100 in² / 645.16 cm²,27in / 68.58cm,11.4oz / 323g,13in / 33.02cm / 4 pts HL,62,23mm / 26mm / 23mm,Graphite,...,Medium-Fast,Blue,Wilson Pro Performance,16 Mains / 19 Crosses\n\n\nMains skip,50-60 pounds,9-10,8.1 ounces / 230 grams,50-55 inches / 127cm-140cm,String Tension: 45-55 pounds\nSolinco recommen...,11.1oz / 315g
freq,2,2,2,153,255,49,49,33,33,118,...,221,55,42,135,87,19,1,1,2,2


In [49]:
# Reset index and drop column of 0s to allow for accuracte index referencing
racquets = racquets.reset_index().drop(columns = "index")
racquets.head()

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Swing Speed:,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.8,289.0,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.0,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.0,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.0,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.0,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# Look at columns that need to be converted
racquets[["Head Size", "Length", "Strung Weight", "Balance",
          "Stiffness", "Beam Width", "String Pattern", "String Tension"]]

,Head Size,Length,Strung Weight,Balance,Stiffness,Beam Width,String Pattern,String Tension
0,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,69,23mm / 26mm / 23mm,16 Mains / 19 CrossesMains skip,46-55 pounds
1,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,69,21mm / 23mm / 21mm,16 Mains / 20 Crosses\n\n\nMains skip,46-55 pounds
2,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,69,21mm / 23mm / 21mm,16 Mains / 20 Crosses\n\n\nMains skip,46-55 pounds
3,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,69,23mm / 26mm / 23mm,16 Mains / 19 CrossesMains skip,46-55 pounds
4,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,69,23mm / 26mm / 23mm,16 Mains / 19 Crosses\n\n\nMains skip,44-53 pounds
...,...,...,...,...,...,...,...,...
392,100 in² / 645.16 cm²,27.5in / 69.85cm,11.3oz / 320g,12.8in / 32.51cm / 8 pts HL,70,23.5mm / 26mm / 23mm,16 Mains / 19 Crosses\n\n\nMains skip,50-60 pounds
393,100 in² / 645.16 cm²,28in / 71.12cm,11.3oz / 320g,12.8in / 32.51cm / 10 pts HL,66,23.5mm / 26mm / 23mm,16 Mains / 19 CrossesMains skip,50-60 pounds
394,100 in² / 645.16 cm²,27in / 68.58cm,11.1oz / 315g,12.9in / 32.77cm / 5 pts HL,69,23mm / 25mm / 23mm,16 Mains / 19 Crosses\n\n\nMains skip,51-55 pounds
395,100 in² / 645.16 cm²,27in / 68.58cm,10.2oz / 289g,13.4in / 34.04cm / 1 pts HL,NaN,23mm / 25mm / 23mm,16 Mains / 19 Crosses\n\n\nMains skip,51-55 pounds


## Looking at how junior racquets affect the data

There are two types of racquets in the data. Regular racquets and junior racquets. My guess is that the junior racquets will have much less informatoin on their specs compared to the regular ones. In the grand context of building a semantic search tool for rackets, I think excluding racquets won't have a noticeable effect on the end product. But, I will still investigate to see if the NA values change noticeably without junior racquets in the data.

In [51]:
# Look at number of "junior" racquets in the data set
racquets["Junior"] = racquets["racquet_name"].str.lower().str.contains("junior")
racquets.head()

,racquet_img,racquet_name,racquet_rating,racquet_price,racquet_desc,Head Size,Length,Strung Weight,Balance,Swingweight,...,Racquet Colors:,Grip Type:,String Pattern:,String Tension:,Age,Weight,Height,Other,Strung Weight,Junior
0,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 2025,4.8,289.0,The Pure Drive is popular for a reason. Boast...,100 in² / 645.16 cm²,27in / 68.58cm,11.2oz / 318g,12.99in / 32.99cm / 4 pts HL,317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2025,4.5,299.0,Originally launched in 2019 under the VS moni...,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,326.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive 98 2-Pack 2025,5.0,579.0,This product is for 2 Pure Drive 98 racquets....,98 in² / 632.26 cm²,27in / 68.58cm,11.4oz / 323g,13.18in / 33.48cm / 3 pts HL,323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Plus 2025,5.0,289.0,Babolat adds another chapter to one of the ga...,100 in² / 645.16 cm²,27.5in / 69.85cm,11.2oz / 318g,13in / 33.02cm / 6 pts HL,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,https://img.tennis-warehouse.com/watermark/rs....,Babolat Pure Drive Team 2025,5.0,269.0,The Pure Drive Team 2025 is defined by its us...,100 in² / 645.16 cm²,27in / 68.58cm,10.6oz / 301g,12.85in / 32.64cm / 5 pts HL,308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


Below, I can clearly see that removing the junior racquets significantly reduces the percentage of NA values in the spec columns. This na table also confirms the duplication issues discussed above. 

In [52]:
# Create a df showing the number of NA values per column, their percentage NA, 
# and their percentage NA when the junior rackets are removed from the data
na_vals = pd.Series(racquets.isna().sum(), name = "nas").to_frame()
na_vals["perc_nas"] = ((na_vals["nas"]/len(racquets)) * 100).round(2)
na_vals["perc_nas_junior_removed"] = (((
    racquets[~racquets["racquet_name"].str.contains("Junior")].isna().sum()
    )/len(
        racquets[~racquets["racquet_name"].str.contains("Junior")]
        )) * 100).round(2)

na_vals.sort_values(by = "perc_nas", ascending = True)

,nas,perc_nas,perc_nas_junior_removed
racquet_img,0,0.00,0.00
racquet_desc,0,0.00,0.00
racquet_price,0,0.00,0.00
Junior,0,0.00,0.00
racquet_name,0,0.00,0.00
Head Size,32,8.06,4.63
Length,32,8.06,4.63
Composition,33,8.31,4.94
Racquet Colors,44,11.08,4.94
racquet_rating,54,13.60,1.54


## Takeaways and next steps

1. Get rid of all junior racquets from df

2. Drop all duplicated columns

3. Add a racquet_brand column by extracting the brand name from racquet_name

4. Convert the following columns from object to float or int using regex or str logic:
    - Head Size
    - Length
    - Strung Weight
    - Balance
        - Create two columns: racquet_balance_in and racquet_balance_HH_HL
    - Stiffness
    - Beam width
    - String Pattern
        - Create two columns: racquet_mains and racquet_crosses
    - String Tension
        - Create two columns: racquet_tension_lower and racquet_tension_upper


Overall, the scraper did a decent job for it being the first one I've ever made! For some reason, the spec columns got duplicated. My best guess is that there was some discrepancies in how they were labelled in the html. There are junior racquets present in the data, which aren't relevant and there are several columns that need to be processed into a different data type. 